In [1]:
from riot_api_utils import riot_api_utils
import pandas as pd
import numpy as np
import json

/Users/cgjeong/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### API Key

In [2]:
api_key = "RGAPI-6eda9521-a7e3-4807-82db-1e3cb0e83bb7"
riot = riot_api_utils.riot_api_utils(api_key)

## C, GM user list

In [3]:
# get challenger, grandmaster list
summoner_id_list = riot.get_challenger_list() + riot.get_grandmaster_list() + riot.get_master_list()
count = 20

# shuffle summoner_id_list
summoner_id_list  = np.random.permutation(summoner_id_list)
# match_id_list as heap queue
match_id_list = []


In [4]:
# file directory check
import os
if not os.path.exists("data"):
    os.makedirs("data")

if not os.path.exists("data/match"):
    os.makedirs("data/match")

if not os.path.exists("data/match_timeline"):
    os.makedirs("data/match_timeline")

In [5]:
# read match_id.csv
df = pd.read_csv("data/match_id_list.csv")
match_id_list = list(df["match_id"])
time_stamp  = list(df["time_stamp"])

In [6]:
from tqdm.notebook import tqdm
p_bar1 = tqdm(summoner_id_list)
p_bar2 = tqdm(range(count))

for summoner_id in p_bar1:
    # get puuid
    puuid = riot.get_puuid(summoner_id['summonerId'])
    # get match list
    match_list = riot.get_match_list(puuid,count)
    if match_list == None:
        print("session expired")
        continue
    
    p_bar2.reset()
    p_bar2.refresh()
    for match_id in match_list:
        p_bar2.set_postfix({"# of data": len(match_id_list)})
        p_bar2.update(1)
        p_bar2.refresh()
        if match_id not in match_id_list:    
            # get match timeline
            match_timeline = riot.get_match_timeline(match_id)
            if match_timeline == None:
                print("session expired")
                break
            # convert to json
            match_tl_json = json.dumps(match_timeline)

            # get match
            match = riot.get_match(match_id)
            if match == None:
                print("session expired")
                break
            # convert to json
            match_json = json.dumps(match)

            # save to file
            with open(f'data/match_timeline/{match_id}.json', 'w') as f:
                f.write(match_tl_json)
                f.close()
            # save to file
            with open(f'data/match/{match_id}.json', 'w') as f:
                f.write(match_json)
                f.close()
            match_id_list.append(match_id)
            time_stamp.append(pd.Timestamp.now())
            #len(match_id_list)
        else:
            continue
    
    df = pd.DataFrame({"match_id":match_id_list, "time_stamp":time_stamp})
    df.to_csv("data/match_id_list.csv",index=False)

  0%|          | 0/6858 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]